In [1]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


# **Pytorch Lightning**

In [2]:
import pytorch_lightning as pl
import torchmetrics
import torch
import torch.nn as nn

# Preparing Dataset & Dataloaders

In [3]:
class DigitDataset:
    
    def __init__(self, data, labels = None, is_test = False):
        self.data = data / 255.0
        self.labels = labels if labels is not None else None
        self.is_test = is_test
    
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        
        sample = self.data[idx,:]
        sample = torch.tensor(sample, dtype=torch.float)
        sample = sample.view(28,28).unsqueeze(0)
        
        if self.is_test is False:
            
            label = self.labels[idx]
            
            return (
                sample,
                torch.tensor(label, dtype=torch.long)
            )
        
        else:
            return sample

In [4]:
train_df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_df = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [5]:
from sklearn.model_selection import train_test_split

# train and validation split
X = train_df.iloc[:,1:].values
y = train_df.iloc[:,0].values
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.2)


# test 
X_test = test_df.values

train_ds = DigitDataset(X_train, y_train)
val_ds = DigitDataset(X_val, y_val)
test_ds = DigitDataset(X_test, is_test = True)

In [26]:
train_dataloader = torch.utils.data.DataLoader(dataset=train_ds, 
                                               batch_size=64, 
                                               num_workers=2, 
                                               shuffle=True
                                              )

val_dataloader = torch.utils.data.DataLoader(dataset=val_ds, 
                                             batch_size=64, 
                                             num_workers=2, 
                                             shuffle=False
                                            )

test_dataloader = torch.utils.data.DataLoader(dataset=test_ds,
                                             batch_size=128,
                                             shuffle=False
                                             )

In [27]:
len(train_dataloader), len(val_dataloader), len(test_dataloader)

(525, 132, 219)

# **Model**

In [34]:
class LightningDigits(pl.LightningModule):
    
    def __init__(self, in_channels = 1, num_classes = 10):
        super().__init__()
        self.in_channels = in_channels
        self.num_classes = num_classes
        
        self.train_acc = torchmetrics.Accuracy()
        self.val_acc = torchmetrics.Accuracy()
        
        self.test_preds = []
        
        self.stack = nn.Sequential(
            nn.Conv2d(self.in_channels,16,3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16,32,3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(32*5*5, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128,self.num_classes)
        )
        
        self.loss_fn = nn.CrossEntropyLoss()
        
    def forward(self, x):
        x = self.stack(x)
        return x
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    
    def training_step(self, batch, batch_idx):
        
        x, y = batch
        
        preds = self.stack(x)
        
        loss = self.loss_fn(preds, y)
        self.train_acc(torch.argmax(preds, dim=1), y)
        
        self.log('train_loss', loss.item(), on_epoch=True)
        self.log('train_acc', self.train_acc, on_epoch=True)
        
        return loss
    
    
    def validation_step(self, batch, batch_idx):
        
        x,y = batch
        
        preds = self.stack(x)
        
        loss = self.loss_fn(preds, y)
        self.val_acc(torch.argmax(preds, dim=1), y)
        
        self.log('val_loss', loss.item(), on_epoch=True)
        self.log('val_acc', self.val_acc, on_epoch=True)
        
    def test_step(self, batch, batch_idx):
        
        x = batch
        
        preds = self.stack(x)
        pred_labels = torch.argmax(preds, dim=1)
        
        self.test_preds.append(pred_labels)
        
    def get_test_preds(self):
        return torch.concat(self.test_preds)

In [35]:
model = LightningDigits()
trainer = pl.Trainer(accelerator='gpu', max_epochs=20)

trainer.fit(model, train_dataloader, val_dataloader)

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

# **Submission**

In [41]:
trainer.test(model, test_dataloader)

Testing: 0it [00:00, ?it/s]

[{}]

In [48]:
test_preds = model.get_test_preds().detach().cpu().numpy()

In [49]:
submission_df = pd.DataFrame({'ImageId': list(range(1,len(test_df)+1)),
                              'Label': test_preds
                             })

submission_df.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [50]:
submission_df.to_csv('submission.csv', index=False)